In [76]:
pip install pandas plotly requests jupyter 

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
from datetime import datetime, timedelta


In [4]:
url="https://disease.sh/v3/covid-19/historical/all?lastdays=30"
response=requests.get(url)
print(response)
data=response.json()
cases_df=pd.DataFrame(list(data["cases"].items()),columns=['Date','Cases'])
deaths_df=pd.DataFrame(list(data['deaths'].items()),columns=['Date','Deaths'])
recovered_df=pd.DataFrame(list(data['recovered'].items()),columns=['Date','Recovered'])
merged_df=pd.merge(cases_df,deaths_df,on='Date')
merged_df=pd.merge(merged_df,recovered_df,on='Date')
merged_df['Date']=pd.to_datetime(merged_df['Date'],format='%m/%d/%y')
fig=make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=merged_df['Date'],y=merged_df['Cases'],name="Cases",line=dict(color='blue')),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=merged_df['Date'],y=merged_df['Deaths'],name='Deaths',line=dict(color='red')),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x=merged_df['Date'],y=merged_df['Recovered'],name="Recovered",line=dict(color='green')),
    secondary_y=False,
)
fig.update_layout(
    title_text="Global COVID-19 Trends (Last 30 Days)",
    hovermode="x unified"
)
fig.update_yaxes(title_text="Cases/Recovered",secondary_y=False)
fig.update_yaxes(title_text="Deaths",secondary_y=True)
fig.show()
countries_url="https://disease.sh/v3/covid-19/countries"
countries_data=requests.get(countries_url).json()
print(countries_data)
countries_df=pd.DataFrame(countries_data)
countries_df=countries_df[['country','cases','deaths','recovered','active','countryInfo']]
countries_df['lat']=countries_df['countryInfo'].apply(lambda x: x['lat'])
countries_df['long']=countries_df['countryInfo'].apply(lambda x: x['long'])
fig=px.scatter_geo(countries_df,
                   lat='lat',
                   lon='long',
                   color="cases",
                   size="cases",
                   hover_name="country",
                   hover_data=["cases","deaths","recovered"],
                   projection="natural earth",
                   title="Global COVID-19 Spread",
                   color_continuous_scale=px.colors.sequential.Reds)
fig.update_layout(height=600,margin={"r":0,"t":30,"l":0,"b":0})
fig.show()
def get_country_historical_data(country):
    url=f"https://disease.sh/v3/covid-19/historical/{country}?lastdates=30"
    response=requests.get(url)
    if response.status_code==200:
        data=response.json()
        timeline=data['timeline']
        df=pd.DataFrame({
            'Date':list(timeline['cases'].keys()),
            'Cases':list(timeline['cases'].values()),
            'Deaths':list(timeline['deaths'].values()),
            'Recovered':list(timeline['recovered'].values())
        })
        df['Date']=pd.to_datetime(df['Date'],format='%m/%d/%y')
        return df
    else:
        return None
india_df=get_country_historical_data('India') 
print(india_df)  
india_df.info()
from ipywidgets import interact,widgets
country_dropdown=widgets.Dropdown(options=sorted(countries_df['country'].unique()),
                                  value='India',
                                  description='Country'
                                  )
from ipywidgets import interact, widgets


country_dropdown = widgets.Dropdown(
    options=sorted(countries_df['country'].unique()),
    value='India',
    description='Country:'
)


data_type_radio = widgets.RadioButtons(
    options=['Cases', 'Deaths', 'Recovered', 'Active'],
    value='Cases',
    description='Data Type:'
)

@interact(country=country_dropdown, data_type=data_type_radio)
def update_plots(country, data_type):
    
    country_df = get_country_historical_data(country)
    
    if country_df is not None:
        
        fig1 = px.line(country_df, x='Date', y=data_type, 
                       title=f"{country} - COVID-19 {data_type} Over Time")
        fig1.show()
        
        
        country_data = countries_df[countries_df['country'] == country].iloc[0]
        fig2 = px.scatter_geo(countries_df, 
                             lat='lat', 
                             lon='long', 
                             size=data_type.lower(),
                             color=data_type.lower(),
                             hover_name="country",
                             projection="natural earth",
                             title=f"Global COVID-19 {data_type}")
        
        
        fig2.add_trace(
            go.Scattergeo(
                lon=[country_data['long']],
                lat=[country_data['lat']],
                text=[country],
                mode='markers',
                marker=dict(size=15, color='black'),
                name='Selected Country'
            )
        )
        
        fig2.update_layout(height=500, margin={"r":0,"t":30,"l":0,"b":0})
        fig2.show()


<Response [200]>


[{'updated': 1751205289399, 'country': 'Afghanistan', 'countryInfo': {'_id': 4, 'iso2': 'AF', 'iso3': 'AFG', 'lat': 33, 'long': 65, 'flag': 'https://disease.sh/assets/img/flags/af.png'}, 'cases': 234174, 'todayCases': 0, 'deaths': 7996, 'todayDeaths': 0, 'recovered': 211080, 'todayRecovered': 0, 'active': 15098, 'critical': 0, 'casesPerOneMillion': 5746, 'deathsPerOneMillion': 196, 'tests': 1390730, 'testsPerOneMillion': 34125, 'population': 40754388, 'continent': 'Asia', 'oneCasePerPeople': 174, 'oneDeathPerPeople': 5097, 'oneTestPerPeople': 29, 'activePerOneMillion': 370.46, 'recoveredPerOneMillion': 5179.32, 'criticalPerOneMillion': 0}, {'updated': 1751205289384, 'country': 'Albania', 'countryInfo': {'_id': 8, 'iso2': 'AL', 'iso3': 'ALB', 'lat': 41, 'long': 20, 'flag': 'https://disease.sh/assets/img/flags/al.png'}, 'cases': 334863, 'todayCases': 0, 'deaths': 3605, 'todayDeaths': 0, 'recovered': 330233, 'todayRecovered': 0, 'active': 1025, 'critical': 0, 'casesPerOneMillion': 116825,

         Date     Cases  Deaths  Recovered
0  2023-02-08  44684973  530748          0
1  2023-02-09  44685087  530750          0
2  2023-02-10  44683862  530750          0
3  2023-02-11  44685219  530750          0
4  2023-02-12  44685425  530753          0
5  2023-02-13  44685499  530753          0
6  2023-02-14  44685601  530756          0
7  2023-02-15  44685727  530757          0
8  2023-02-16  44685884  530757          0
9  2023-02-17  44686001  530757          0
10 2023-02-18  44686143  530760          0
11 2023-02-19  44686263  530761          0
12 2023-02-20  44686358  530761          0
13 2023-02-21  44686483  530762          0
14 2023-02-22  44686676  530763          0
15 2023-02-23  44686842  530764          0
16 2023-02-24  44687025  530764          0
17 2023-02-25  44687243  530769          0
18 2023-02-26  44687428  530770          0
19 2023-02-27  44687597  530771          0
20 2023-02-28  44687837  530772          0
21 2023-03-01  44688105  530772          0
22 2023-03-

interactive(children=(Dropdown(description='Country:', index=94, options=('Afghanistan', 'Albania', 'Algeria',…